In [1]:
import json
import requests
import pandas as pd
import os
import time

def get_cik_data():
    """
    ดึงข้อมูล CIK และ Ticker ของบริษัทต่างๆ จาก SEC
    """
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    # **สำคัญ:** เปลี่ยน YourAppName และ YourContactEmail@example.com เป็นข้อมูลของคุณ
    # การระบุ User-Agent ที่เหมาะสมเป็นสิ่งจำเป็นตามข้อกำหนดของ SEC EDGAR
    headers = {'User-Agent': 'AgentIdea (Contact: chanathip.thc@gmail.com)'}
    try:
        print("กำลังดึงข้อมูลบริษัทจาก SEC...")
        resp = requests.get(url, headers=headers)
        resp.raise_for_status() # ตรวจสอบว่า request สำเร็จหรือไม่
        data = resp.json()
        print("ดึงข้อมูลสำเร็จ กำลังประมวลผล...")

        fields = data.get('fields', [])
        try:
            # พยายามหา index ของ 'cik' หรือ 'cik_str'
            cik_index = fields.index('cik')
        except ValueError:
            try:
                cik_index = fields.index('cik_str')
            except ValueError:
                print("!!! ข้อผิดพลาด: ไม่พบฟิลด์ 'cik' หรือ 'cik_str' ในข้อมูล JSON จาก SEC")
                return None

        try:
            # พยายามหา index ของ 'ticker'
            ticker_index = fields.index('ticker')
        except ValueError:
            print("!!! คำเตือน: ไม่พบฟิลด์ 'ticker' ในข้อมูล JSON จาก SEC อาจทำให้ไม่สามารถดึงข้อมูลจาก FMP API ได้")
            ticker_index = -1 # ตั้งค่าเป็น -1 หากไม่พบคอลัมน์ ticker

        company_data = data.get('data', [])
        records = []
        processed_count = 0
        skipped_count = 0

        for item in company_data:
            cik = None
            ticker = None
            try:
                # ดึงและจัดรูปแบบ CIK ให้เป็น 10 หลัก
                if cik_index != -1 and len(item) > cik_index:
                    cik_raw = item[cik_index]
                    # แปลงเป็น int ก่อนเพื่อให้ zfill ทำงานกับตัวเลขได้ถูกต้อง แล้วค่อยแปลงกลับเป็น str
                    if isinstance(cik_raw, (int, str)) and str(cik_raw).isdigit():
                         cik = str(int(cik_raw)).zfill(10)
                    else:
                         # print(f"ข้ามรายการเนื่องจาก CIK ไม่ใช่ตัวเลข: {item}")
                         skipped_count += 1
                         continue


                # ดึง Ticker
                if ticker_index != -1 and len(item) > ticker_index:
                    ticker = item[ticker_index]

                # เพิ่มเฉพาะรายการที่มี Ticker (จำเป็นสำหรับ FMP API)
                # และ CIK ที่ถูกต้อง
                if cik and ticker:
                    records.append({'cik_str': cik, 'ticker': ticker})
                    processed_count += 1
                else:
                    # print(f"ข้ามรายการเนื่องจากไม่มี Ticker หรือ CIK: {item}")
                    skipped_count += 1


            except (TypeError, IndexError, ValueError) as e:
                 # print(f"ข้ามข้อมูลบางรายการเนื่องจากข้อผิดพลาดในการประมวลผล: {item} - {e}")
                 skipped_count += 1
                 continue # ข้ามรายการที่มีปัญหา

        print(f"ประมวลผลข้อมูลบริษัท: พบ {processed_count} รายการที่สมบูรณ์, ข้าม {skipped_count} รายการ")

        if not records:
            print("!!! ไม่พบข้อมูลบริษัทที่ถูกต้อง (มีทั้ง CIK และ Ticker)")
            return None

        df = pd.DataFrame(records)
        # ลบ Ticker ที่ซ้ำกัน อาจจะเก็บ CIK แรกที่เจอไว้ (ถ้าต้องการ)
        # df = df.drop_duplicates(subset=['ticker'], keep='first')
        # print(f"จำนวนบริษัทหลังจากลบ Ticker ซ้ำ: {len(df)}")
        return df.drop_duplicates(subset=['ticker'], keep='first') # คืนค่า DataFrame ที่ไม่มี Ticker ซ้ำ

    except requests.exceptions.RequestException as e:
        print(f"!!! เกิดข้อผิดพลาดในการดึงข้อมูล JSON จาก SEC: {e}")
        return None
    except Exception as e:
        print(f"!!! เกิดข้อผิดพลาดโดยรวมใน get_cik_data: {e}")
        return None

def download_financial_statements_api(ticker, cik, api_key):
    """
    ดาวน์โหลด financial statements (Annual) ผ่าน API ของ Financial Modeling Prep
    และบันทึกเป็นไฟล์ CSV
    """
    if not ticker:
        # โดยปกติ โค้ดส่วนหลักจะกรอง Ticker ที่เป็น None ออกไปแล้ว แต่ใส่ไว้เผื่อ
        print(f"ข้าม CIK: {cik} เนื่องจากไม่มี Ticker Symbol")
        return

    base_api_url = "https://financialmodelingprep.com/api/v3/"
    # รายการ statement ที่ต้องการดึงข้อมูล
    statements = {
        "Income_Statement": f"{base_api_url}income-statement/{ticker}",
        "Balance_Sheet": f"{base_api_url}balance-sheet-statement/{ticker}",
        "Cash_Flow_Statement": f"{base_api_url}cash-flow-statement/{ticker}"
    }
    output_folder = "downloaded_csvs_api" # ชื่อโฟลเดอร์สำหรับเก็บไฟล์ CSV
    os.makedirs(output_folder, exist_ok=True) # สร้างโฟลเดอร์ถ้ายังไม่มี

    # พารามิเตอร์สำหรับส่งไปกับ API request
    params = {
        "period": "annual",  # หรือ "quarterly" ถ้าต้องการรายไตรมาส
        "limit": 10,         # ดาวน์โหลดข้อมูลย้อนหลังกี่ปี (ปรับตามต้องการ)
        "apikey": api_key    # API Key ของคุณ
    }

    # **สำคัญ:** ระบุ User-Agent ใน Header ด้วย
    headers = {'User-Agent': 'AgentIdea (Contact: chanathip.thc@gmail.com)'}

    print(f"--- กำลังดึงข้อมูลสำหรับ Ticker: {ticker} (CIK: {cik}) ---")
    for name, url in statements.items():
        try:
            # ส่ง GET request ไปยัง API
            response = requests.get(url, params=params, headers=headers, timeout=30) # เพิ่ม timeout
            response.raise_for_status()  # เช็ค HTTP status code (ถ้าไม่ใช่ 2xx จะเกิด exception)

            data = response.json() # แปลง JSON response เป็น Python object

            # ตรวจสอบว่า API ส่งข้อมูลกลับมาเป็น list และมีข้อมูลข้างในหรือไม่
            if isinstance(data, list) and len(data) > 0:
                df = pd.DataFrame(data) # แปลง list of dictionaries เป็น DataFrame

                # --- (Optional) การปรับแต่ง DataFrame ก่อนบันทึก ---
                # เช่น เปลี่ยนชื่อคอลัมน์, เลือกเฉพาะคอลัมน์, จัดรูปแบบวันที่
                # df['date'] = pd.to_datetime(df['date'])
                # df = df.set_index('date')
                # ----------------------------------------------------

                # สร้างชื่อไฟล์: Ticker_StatementName_Period.csv
                filename = f"{ticker}_{name}_annual.csv"
                output_path = os.path.join(output_folder, filename)

                # บันทึก DataFrame เป็นไฟล์ CSV
                df.to_csv(output_path, index=False) # index=False คือไม่บันทึก index ของ DataFrame ลงในไฟล์
                print(f"  [สำเร็จ] บันทึก {name} -> {output_path}")

            # ตรวจสอบกรณี API ส่งข้อความ Error กลับมา (บาง API ทำแบบนี้)
            elif isinstance(data, dict) and 'Error Message' in data:
                 print(f"  [API Error] {name}: {data['Error Message']}")
            # กรณีอื่นๆ ที่ไม่ได้รับข้อมูลที่คาดหวัง
            else:
                print(f"  [ไม่พบข้อมูล] ไม่ได้รับข้อมูล {name} หรือรูปแบบไม่ถูกต้อง")
                # print(f"    Data received: {data}") # Uncomment เพื่อดูข้อมูลที่ได้รับจริง

            # หน่วงเวลาเล็กน้อยก่อนเรียก API ครั้งถัดไป ป้องกัน Rate Limit
            time.sleep(0.5) # ลดเวลาลงเล็กน้อย ถ้าไม่ติด Rate Limit

        except requests.exceptions.HTTPError as e:
            # จัดการข้อผิดพลาด HTTP เช่น 401 Unauthorized (API Key ผิด), 403 Forbidden, 404 Not Found, 429 Too Many Requests
             print(f"  [HTTP Error] {name} ({e.response.status_code}): {e}")
             try:
                 error_details = e.response.json()
                 print(f"    Details: {error_details}")
             except json.JSONDecodeError:
                 print(f"    Response Body: {e.response.text}")
             if e.response.status_code == 429:
                 print("    !!! อาจถึงขีดจำกัดการเรียก API (Rate Limit) กำลังรอสักครู่...")
                 time.sleep(10) # รอ 10 วินาทีถ้าเจอ Rate Limit
        except requests.exceptions.Timeout:
            print(f"  [Timeout Error] {name}: การร้องขอหมดเวลา")
            time.sleep(5) # รอสักครู่ก่อนลองใหม่ (ถ้ามีการวนซ้ำ)
        except requests.exceptions.RequestException as e:
            # ข้อผิดพลาดอื่นๆ เกี่ยวกับการเชื่อมต่อ/request
            print(f"  [Request Error] {name}: {e}")
        except Exception as e:
            # ข้อผิดพลาดอื่นๆ ที่ไม่คาดคิดระหว่างประมวลผล
            print(f"  [Unexpected Error] ในการประมวลผล {name}: {e}")
        # finally:
            # อาจจะใส่ time.sleep() ใน finally เพื่อให้มีการหน่วงเสมอ ไม่ว่าจะสำเร็จหรือ error
            # time.sleep(1)


# --- ส่วนหลักในการรันสคริปต์ ---
if __name__ == "__main__":
    # ==================================================
    #  !!! กรุณาใส่ API Key ของคุณที่นี่ !!!
    # ==================================================
    FMP_API_KEY = "TTBEw02NHIVS6EbcDiOOQd5rewsoQCoN"  # <--- แทนที่ด้วย API Key จริงจาก Financial Modeling Prep

    # ตรวจสอบว่าใส่ API Key หรือยัง
    if not FMP_API_KEY or FMP_API_KEY == "YOUR_ACTUAL_FMP_API_KEY_HERE":
        print("="*60)
        print("!!! คำเตือน: กรุณาใส่ API Key ของคุณในตัวแปร FMP_API_KEY")
        print("!!! คุณสามารถรับ API Key ได้จากเว็บไซต์ Financial Modeling Prep")
        print("="*60)
    else:
        # 1. ดึงข้อมูล CIK และ Ticker จาก SEC
        cik_df = get_cik_data()

        if cik_df is not None and not cik_df.empty:
            print("\nข้อมูล CIK และ Ticker ที่ดึงมา (ตัวอย่าง 5 รายการแรก):")
            print(cik_df.head())
            print(f"\nพบ Ticker Symbol ที่ไม่ซ้ำกันทั้งหมด {len(cik_df)} รายการ")

            print(f"\n===== เริ่มดาวน์โหลด Financial Statements ผ่าน API =====")
            print(f"จะดาวน์โหลดข้อมูลสำหรับ {len(cik_df)} บริษัท...")
            print(f"ไฟล์จะถูกบันทึกในโฟลเดอร์: '{os.path.abspath('downloaded_csvs_api')}'")


            # --- (Optional) เลือกดาวน์โหลดเฉพาะบางส่วนเพื่อทดสอบ ---
            # start_index = 0
            # end_index = 10 # ดาวน์โหลดแค่ 10 บริษัทแรก
            # cik_df_subset = cik_df.iloc[start_index:end_index]
            # print(f"\n!!! กำลังทดสอบดาวน์โหลดเฉพาะ {len(cik_df_subset)} บริษัทแรก !!!")
            # loop_target = cik_df_subset.iterrows()
            # ----------------------------------------------------

            loop_target = cik_df.iterrows() # ใช้ DataFrame ทั้งหมด

            # 2. วนลูปเพื่อดาวน์โหลดข้อมูลสำหรับแต่ละบริษัท
            total_companies = len(cik_df) # ใช้ len(cik_df_subset) ถ้าทดสอบ
            for index, row in loop_target:
                current_index = index + 1 # สำหรับแสดงลำดับ (ถ้าใช้ df ทั้งหมด)
                # ถ้าใช้ subset ต้องปรับ index เป็น index - start_index + 1
                # current_index = index - start_index + 1
                cik = row['cik_str']
                ticker = row['ticker']

                print(f"\n({current_index}/{total_companies}) บริษัท: {ticker} (CIK: {cik})")
                download_financial_statements_api(ticker, cik, FMP_API_KEY)

                # อาจเพิ่มการหน่วงเวลาระหว่างบริษัท หากเจอปัญหา Rate Limit บ่อยๆ
                # time.sleep(2) # รอ 2 วินาทีก่อนทำบริษัทถัดไป

            print("\n===== การดำเนินการดาวน์โหลด Financial Statements ผ่าน API เสร็จสิ้น =====")

        elif cik_df is not None and cik_df.empty:
             print("\nไม่พบข้อมูลบริษัทที่ถูกต้อง (มี CIK และ Ticker) หลังจากประมวลผลข้อมูลจาก SEC")
        else:
            # กรณี get_cik_data() คืนค่า None
            print("\n!!! ไม่สามารถดำเนินการต่อได้ เนื่องจากไม่สามารถดึงข้อมูล CIK และ Ticker ได้")

กำลังดึงข้อมูลบริษัทจาก SEC...
ดึงข้อมูลสำเร็จ กำลังประมวลผล...
ประมวลผลข้อมูลบริษัท: พบ 9691 รายการที่สมบูรณ์, ข้าม 0 รายการ

ข้อมูล CIK และ Ticker ที่ดึงมา (ตัวอย่าง 5 รายการแรก):
      cik_str ticker
0  0000320193   AAPL
1  0000789019   MSFT
2  0001045810   NVDA
3  0001652044  GOOGL
4  0001018724   AMZN

พบ Ticker Symbol ที่ไม่ซ้ำกันทั้งหมด 9691 รายการ

===== เริ่มดาวน์โหลด Financial Statements ผ่าน API =====
จะดาวน์โหลดข้อมูลสำหรับ 9691 บริษัท...
ไฟล์จะถูกบันทึกในโฟลเดอร์: 'C:\Users\Gusju\Webscrap\downloaded_csvs_api'

(1/9691) บริษัท: AAPL (CIK: 0000320193)
--- กำลังดึงข้อมูลสำหรับ Ticker: AAPL (CIK: 0000320193) ---
  [สำเร็จ] บันทึก Income_Statement -> downloaded_csvs_api\AAPL_Income_Statement_annual.csv
  [สำเร็จ] บันทึก Balance_Sheet -> downloaded_csvs_api\AAPL_Balance_Sheet_annual.csv
  [สำเร็จ] บันทึก Cash_Flow_Statement -> downloaded_csvs_api\AAPL_Cash_Flow_Statement_annual.csv

(2/9691) บริษัท: MSFT (CIK: 0000789019)
--- กำลังดึงข้อมูลสำหรับ Ticker: MSFT (CIK: 0000789019) -

KeyboardInterrupt: 